In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_openml
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

data= fetch_openml('mnist_784', version=1)#Get data from https://www.openml.org/d/554
dfData = pd.DataFrame(np.c_[data["data"],data["target"]],columns = data["feature_names"]+["target"])

In [2]:
img_pipeline = Pipeline([("mm_scaler", MinMaxScaler())])
y = dfData["target"]
dfData = dfData.drop("target",axis=1)
X = dfData.copy()
X_transf = img_pipeline.fit_transform(X)

In [3]:
stratSplit = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
for train_index, test_index in stratSplit.split(X_transf, y):
    X_train = X_transf[train_index]
    X_test = X_transf[test_index]
    y_train = y[train_index].values
    y_test = y[test_index].values

In [4]:
# turning off warnings since we'll always be pushing the iteration limit here
# https://stackoverflow.com/questions/32612180/eliminating-warnings-from-scikit-learn
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

iterTrainSampleSize = 5000
def getIterTrainSplit(X, y, iterTrainSampleSize):
    test_size = 1-iterTrainSampleSize/X.shape[0]
    iterStratSplit = StratifiedShuffleSplit(n_splits=1, test_size=test_size)
    for train_index, test_index in iterStratSplit.split(X, y):
        return X[train_index], y[train_index], X[test_index], y[test_index]
    
    

log_reg = LogisticRegression(max_iter=1,
                             warm_start=True,
                             verbose=False)# run model training one iteration at a tame
trainingPerformance = []
valPerformance = []
iteration = []
for i in range(200):
    X_train_iter, y_train_iter, X_val_iter, y_val_iter = getIterTrainSplit(X_train, y_train, iterTrainSampleSize)
    log_reg.fit(X_train_iter, y_train_iter)
    trainingPerformance.append(log_reg.score(X_train_iter, y_train_iter))
    valPerformance.append(log_reg.score(X_val_iter, y_val_iter))
    iteration.append(i+1)

plt.plot(iteration,trainingPerformance,color='orange',label='train performance')
plt.plot(iteration,valPerformance,color='green',label='validation performance')
plt.legend()
plt.title('Train and validation performance across training iterations for log. reg.')
plt.xlabel('Training Iteration')
plt.ylabel('Acccuracy sccore')
plt.ylim(0,1)
plt.show()

In [5]:
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.learning_curve.html
# Helpful documentation example:
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html#sphx-glr-auto-examples-model-selection-plot-learning-curve-py
from sklearn.model_selection import learning_curve

training_sizes, train_scores, test_scores = learning_curve(LogisticRegression(),
                   X_train,
                   y_train,
                   train_sizes=np.linspace(0.01, 1.0, 50),
                   shuffle=True
                  )
plt.plot(np.linspace(0.01*X_train.shape[0],X_train.shape[0],50),
         np.mean(train_scores,axis=1),
         color='orange',
         label='train performance')
plt.plot(np.linspace(0.01*X_train.shape[0],X_train.shape[0],50),
         np.mean(test_scores,axis=1),
         color='green',
         label='validation performance')
plt.legend()
plt.title('Training sample size vs performance')
plt.xlabel('Training sample size')
plt.ylabel('Performance')
plt.show()